In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import plotly.express as px

url = "https://www.arrowcast.net/fids/mco/fids.asp?sort=city&city=&number=&airline=&adi=d"
response = requests.get(url)

flight_info = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    flights_table = soup.find("tbody")  

    if flights_table:
        for row in flights_table.find_all("tr"):
            cells = row.find_all("td")
            if len(cells) == 9:
                new_flight_info = [cell.get_text(strip=True) for cell in cells]
                flight_info.append(new_flight_info)
            else:
                print(f"Found a row with {len(cells)} cells. Skipping.")
    else:
        print("Could not find table.")
else:
    print(f"Error code: {response.status_code}")

columns = ["Logo", 
           "Airline",
            "Flight",
            "City",
            "Scheduled Time",
            "Actual Time",
            "Status", 
            "Claim/Terminal",
            "Gate"]
df = pd.DataFrame(flight_info, columns=columns)
df.drop(columns=["Logo"], inplace=True)
df["Scheduled Time"] = pd.to_datetime(df["Scheduled Time"],errors='coerce', format='%a %H:%M')
df["Actual Time"] = pd.to_datetime(df["Actual Time"],errors='coerce', format='%a %H:%M')

Found a row with 1 cells. Skipping.


In [87]:
cmap = {
    'Departed': '#098afc',
    'Cancelled': '#fc3109',
    'On Time': '#0fc74f',
    'Delayed': '#ffb600',
}
color_palette = list(cmap.values())

df["Status"] = df["Status"].apply(lambda x: "Delayed" if "Now" in x else x)
filtered_df = df[df["Status"].isin(["Cancelled", "Departed", "On Time", "Delayed"])]
flights_by_airline = filtered_df.groupby(['Airline', 'Status']).size().reset_index(name='Count').sort_values('Count', ascending=False)
fig_flights = px.bar(flights_by_airline, 
                     y="Count", 
                     x="Airline", 
                     color="Status", 
                     title='Live Departures by Airline', 
                     color_discrete_sequence=color_palette, 
                     category_orders={"Status": list(cmap.keys())},
                     width=900, 
                     height=560
                     )
fig_flights.show()